# Sora2 Watermark Remover - Simple Version

シンプルな6ステップでSora動画からウォーターマークを除去します。

**リポジトリ:** https://github.com/fulfulggg/Sora2WatermarkRemover

## 1. 依存関係のインストール

In [63]:
%pip install -q --upgrade pip wheel setuptools
%pip install -q --prefer-binary --no-build-isolation "iopaint==1.2.2" "transformers==4.42.4" "diffusers==0.26.3" "huggingface_hub==0.24.6" "opencv-python-headless==4.9.0.80" "numpy<2" loguru tqdm

## 2. LaMaモデルのダウンロード

In [64]:
!iopaint download --model lama

2025-10-15 08:52:06.672 | INFO     | iopaint.runtime:setup_model_dir:82 - Model directory: /root/.cache
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is depr

## 3. 最新コードの取得

In [65]:
!curl -sL https://github.com/fulfulggg/Sora2WatermarkRemover/archive/refs/heads/main.tar.gz | tar xz --strip-components=1
print("\n✅ リポジトリのダウンロードが完了しました。")


✅ リポジトリのダウンロードが完了しました。


## 4. 動画のアップロード

In [66]:
from google.colab import files

print("動画ファイルをアップロードしてください:")
uploaded = files.upload()

input_file = list(uploaded.keys())[0]
# 絶対パスに正規化
import os, shutil
src = os.path.abspath(input_file)
_, ext = os.path.splitext(src)
input_path = f"/content/input_uploaded{ext or '.mp4'}"
if src != input_path:
    shutil.move(src, input_path)
print(f"\n✅ アップロード完了: {input_path}")

動画ファイルをアップロードしてください:


Saving slider.mp4 to slider.mp4

✅ アップロード完了: /content/input_uploaded.mp4


## 5. ウォーターマークの除去

Florence-2 AIがウォーターマークを自動検出して除去します（最強設定）。

In [67]:
!python remwm.py "{input_path}" "/content/out.mp4" --overwrite \
  --max-bbox-percent 20 --dilate-px 8 --temporal-window 11 \
  --white-s-max 139 --white-v-min 150 --force-format MP4 --frame-step 1 --target-fps 0
  # --max-bbox-percent 35 --dilate-px 12 --temporal-window 31 \
  # --white-s-max 150 --white-v-min 150 --force-format MP4 --frame-step 1 --target-fps 0

2025-10-15 08:52:41.499853: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-15 08:52:41.516908: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760518361.537894  155303 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760518361.544288  155303 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760518361.560361  155303 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## 6. 結果のダウンロード

In [68]:
print("結果をダウンロード中...")
files.download('/content/out.mp4')
print("\n✅ 完了！")

結果をダウンロード中...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ 完了！
